### Bias Detection and Fairness Evaluation on Heart Failure Prediction Dataset (Kaggle) using FairMLhealth
(Source: https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction/data)

In [1]:
import pandas as pd

# Load X_test set
X_test = pd.read_csv("./data_splits/X_test.csv")
y_test = pd.read_csv("./data_splits/y_test.csv")

In [2]:
import fairmlhealth
import aif360
print("Environment setup successful")

Environment setup successful


In [3]:
#have a look at the details of fairmlhealth - especially the version
!pip show fairmlhealth

Name: fairmlhealth
Version: 1.0.2
Summary: Health-centered variation analysis
Home-page: https://github.com/KenSciResearch/fairMLHealth
Author: Christine Allen
Author-email: ca.magallen@gmail.com
License: 
Location: c:\users\patri\appdata\roaming\python\python310\site-packages
Requires: aif360, ipython, jupyter, numpy, pandas, requests, scikit-learn, scipy
Required-by: 


In [4]:
#have a look at the modules that are within fairmlhealth

print(dir(fairmlhealth))

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__']


In [5]:
#load necessary modules 

#import module measure to use measure.summary for bias detection
from fairmlhealth import measure

#import module for investigation of individual cohorts 
from fairmlhealth.__utils import iterate_cohorts

#import FairRanges to flag high values
from fairmlhealth.__utils import FairRanges

# Wrap the fairness summary function for cohort-wise analysis
@iterate_cohorts
def cohort_summary(**kwargs):
    return measure.summary(**kwargs)

pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\inFairness\utils\ndcg.py:37: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration guide for more details https://pytorch.org/docs/main/func.migrating.html
  vect_normalized_discounted_cumulative_gain = vmap(
C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\inFairness\utils\ndcg.py:48: FutureWarning: We've integrated functorch into PyTorch. As the final step of the integration, `functorch.vmap` is deprecated as of PyTorch 2.0 and will be deleted in a future version of PyTorch >= 2.3. Please use `torch.vmap` instead; see the PyTorch 2.0 release notes and/or the `torch.func` migration 

During the execution of FairMLHealth and AIF360, several runtime warnings were raised (e.g., “AdversarialDebiasing will be unavailable” due to the absence of TensorFlow, and deprecation warnings from the inFairness package regarding PyTorch’s functorch.vmap). These warnings do not affect the fairness metrics or results presented in this study, as the unavailable components were not used. To maintain clarity of output, the warnings were silenced programmatically, and the analysis was conducted without issue.

In [6]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", module="inFairness")
warnings.filterwarnings("ignore", message="AdversarialDebiasing will be unavailable")

### Traditional Machine Learning Models - KNN & DT

#### K-nearest neighbors - KNN

In [7]:
import pandas as pd

# Load KNN results
knn_df = pd.read_csv("HeartFailureData_75M25F_PCA_KNN_predictions.csv")

print(knn_df.head())

   Sex  y_true    y_prob  y_pred
0    1       1  0.810107       1
1    1       1  0.720647       1
2    1       1  0.906213       1
3    1       1  0.728721       1
4    0       0  0.478459       0


In [8]:
# Extract common columns
y_true_knn = knn_df["y_true"].values
y_prob_knn = knn_df["y_prob"].values
y_pred_knn = knn_df["y_pred"].values
gender_knn = knn_df["Sex"].values

# Use gender_knn as the protected attribute (0/1 as in your CSV)
protected_attr_knn = gender_knn

In [9]:
knn_bias = measure.summary(
    X=X_test,
    y_true=y_true_knn,
    y_pred=y_pred_knn,
    y_prob=y_prob_knn,
    prtc_attr=protected_attr_knn,
    pred_type="classification",
    priv_grp=1,
    sig_fig=4,
    skip_if=True,   # skip inconsistency metrics that cause NearestNeighbors error
    skip_performance=True
)

print(knn_bias)

                                                        Value
Metric         Measure                                       
Group Fairness AUC Difference                          0.0497
               Balanced Accuracy Difference           -0.0385
               Balanced Accuracy Ratio                 0.9568
               Disparate Impact Ratio                  0.3842
               Equal Odds Difference                  -0.0521
               Equal Odds Ratio                        1.2500
               Positive Predictive Parity Difference  -0.3889
               Positive Predictive Parity Ratio        0.5882
               Statistical Parity Difference          -0.3796
Data Metrics   Prevalence of Privileged Class (%)     79.0000


In [10]:
# 2) Custom scenario oriented bounds

custom_ranges = {
    "tpr diff": (-0.03, 0.03),
    "fpr diff": (-0.03, 0.03),
    "equal odds difference": (-0.04, 0.04),
    "statistical parity difference": (-0.05, 0.05),
    "disparate impact ratio": (0.9, 1.1),
    "selection ratio": (0.9, 1.1),
    "auc difference": (-0.02, 0.02),
    "balanced accuracy difference": (-0.02, 0.02),
}

bounds = FairRanges().load_fair_ranges(custom_ranges=custom_ranges)

In [11]:
#for highlighting metrics outside of the thresholds
#  restore Styler.set_precision to adjust the highlighting color in the styled table
import pandas as pd, numpy as np

Styler = type(pd.DataFrame({"_":[0]}).style)  

if not hasattr(Styler, "set_precision"):
    def _set_precision(self, precision=4):
        try:
            return self.format(precision=precision)
        except TypeError:
            return self.format(formatter=lambda x:
                f"{x:.{precision}g}" if isinstance(x, (int, float, np.floating)) else x
            )
    setattr(Styler, "set_precision", _set_precision)

In [12]:
#Flag metrics outside acceptable fairness bounds in current table 

from fairmlhealth.__utils import Flagger

class MyFlagger(Flagger):
    def reset(self):
        super().reset()
        self.flag_color = "#491ee6"   
        self.flag_type = "background-color"

styled_knn = MyFlagger().apply_flag(
    df=knn_bias,
    caption="KNN Fairness (Gender)",
    boundaries=bounds,
    sig_fig=4,
    as_styler=True
)
styled_knn

## Gender Bias Detection Results for KNN Model  

---

### Group Fairness Metrics  

- **AUC Difference (0.0497):** A small gap in ranking performance exists, with slightly lower ROC-AUC for one gender group.  
- **Balanced Accuracy Difference (-0.0385)** and **Ratio (0.9568):** Indicate a mild disparity, where females achieve lower balanced accuracy than males.  
- **Disparate Impact Ratio (0.3842):** Well below the acceptable range of 0.80–1.25, showing **strong inequality in selection rates**, with females being much less likely to receive positive predictions.  
- **Equal Odds Difference (-0.0521)** and **Equal Odds Ratio (1.2500):** Suggest some disparity in error rates (TPR/FPR), though less severe than other metrics.  
- **Positive Predictive Parity Difference (-0.3889)** and **Ratio (0.5882):** Precision is significantly lower for females, making their positive predictions much less reliable.  
- **Statistical Parity Difference (-0.3796):** Confirms a strong imbalance, with females being systematically under-selected compared to males.  

---

### Interpretation  

- The KNN model exhibits **serious fairness concerns**:  
  - Females experience **lower predictive quality** (balanced accuracy, precision).  
  - **Selection rates** are highly imbalanced, as shown by the disparate impact and statistical parity metrics.  
  - Error-rate disparities (equal odds) are present but less extreme than the gaps in outcome allocation and predictive reliability.  

---

### **Summary**  

The KNN model shows **systematic gender bias**, disproportionately disadvantaging females (unprivileged group) while favoring males (privileged group).  
The largest issues are found in **selection fairness (disparate impact, statistical parity)** and **prediction reliability (positive predictive parity)**.  
These results suggest that KNN, in its current form, is **not a fair model** and requires mitigation before use in decision-making contexts.  

---

In [13]:
print("FairMLHealth Stratified Bias Table - KNN")
measure.bias(X_test, y_test, y_pred_knn, features=['Sex'], flag_oor=False)

FairMLHealth Stratified Bias Table - KNN


,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,Sex,0,0.0385,1.0451,-0.025,0.80,0.3889,1.7000,0.3796,2.6027,0.0521,1.0625
1,Sex,1,-0.0385,0.9568,0.025,1.25,-0.3889,0.5882,-0.3796,0.3842,-0.0521,0.9412


## Stratified Bias Analysis – KNN by Gender  

This table presents **group-specific fairness metrics** for the KNN model, stratified by gender.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.  

---

### 1. Balanced Accuracy  
- **Females (0):** Balanced Accuracy Difference = **+0.0385**, Ratio = **1.0451**  
- **Males (1):** Balanced Accuracy Difference = **−0.0385**, Ratio = **0.9568**  
- ➝ The model is **slightly more balanced and accurate for females**, while males are at a minor disadvantage.  

---

### 2. False Positive Rate (FPR)  
- **Females (0):** FPR Difference = **−0.0250**, Ratio = **0.80**  
- **Males (1):** FPR Difference = **+0.0250**, Ratio = **1.25**  
- ➝ Females face a **lower false positive rate**, while males are more often incorrectly classified as positive.  

---

### 3. Positive Predictive Value (PPV / Precision)  
- **Females (0):** PPV Difference = **+0.3889**, Ratio = **1.7000**  
- **Males (1):** PPV Difference = **−0.3889**, Ratio = **0.5882**  
- ➝ Predictions are **far more reliable for females**, while males experience substantially lower precision.  

---

### 4. Selection Rate  
- **Females (0):** Selection Difference = **+0.3796**, Ratio = **2.6027**  
- **Males (1):** Selection Difference = **−0.3796**, Ratio = **0.3842**  
- ➝ Females are **selected much more often** for positive predictions, while males are under-selected.  

---

### 5. True Positive Rate (TPR / Sensitivity)  
- **Females (0):** TPR Difference = **+0.0521**, Ratio = **1.0625**  
- **Males (1):** TPR Difference = **−0.0521**, Ratio = **0.9412**  
- ➝ Females have a **slight advantage in sensitivity**, meaning their true cases are more consistently detected compared to males.  

---

### **Summary**  
- The KNN model shows a **reverse bias**:  
  - **Females (unprivileged group)** benefit from higher balanced accuracy, lower false positives, stronger precision, much higher selection rates, and slightly higher sensitivity.  
  - **Males (privileged group)** are disadvantaged across nearly all metrics, with higher false positives, lower precision, under-selection, and slightly weaker sensitivity.  

- Although females gain favorable outcomes, the **magnitude of disparities (especially in PPV and selection rate)** suggests that the KNN model introduces fairness concerns by systematically **favoring females at the expense of males**.  

---

In [14]:
from fairmlhealth import measure
import pandas as pd
from IPython.display import display  

# convert gender into DataFrame with a clear column name to get a nice table as output
gender_df = pd.DataFrame({"Sex": X_test["Sex"].astype(int)})


# Get the stratified table
perf_table_knn = measure.performance(
    X=gender_df,
    y_true=y_test,
    y_pred=y_pred_knn,
    y_prob=y_prob_knn
)

# Replace NaN with a dash
perf_table_knn = perf_table_knn.fillna("—")

# display pretty table
display(perf_table_knn)

C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\fairmlhealth\__utils.py:57: UserWarning: Possible error in column(s) ['Sex']. DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,Feature Name,Feature Value,Obs.,Mean Target,Mean Prediction,Accuracy,F1-Score,FPR,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,184.0,0.5543,0.5380,0.8859,0.8955,0.1098,—,0.9091,0.9253,0.8824
1,Sex,0,38.0,0.1579,0.2368,0.8684,0.6667,0.1250,—,0.5556,0.9583,0.8333
2,Sex,1,146.0,0.6575,0.6164,0.8904,0.9140,0.1000,—,0.9444,0.9086,0.8854


## Stratified Performance Analysis – KNN by Gender  

This table shows the **stratified performance metrics** of the KNN model across gender groups.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.  

---

### 1. Overall Performance (All Features)  
- **Accuracy (0.8859)** and **F1-score (0.8955)** indicate good overall performance.  
- **ROC AUC (0.9253)** demonstrates strong discriminatory ability.  
- **Precision (0.9091)** is high, meaning predictions are generally reliable.  
- **TPR (0.8824)** shows strong sensitivity overall.  
- **Note:** PR AUC is not available (“—”) in this report.  

---

### 2. Subgroup Comparison  

| Metric        | Female (0) | Male (1) | Interpretation |
|---------------|------------|----------|----------------|
| **Accuracy**  | 0.8684     | 0.8904   | Accuracy is slightly higher for males. |
| **F1-Score**  | 0.6667     | 0.9140   | Performance is much stronger for males; females face lower F1. |
| **FPR**       | 0.1250     | 0.1000   | Females have more false positives. |
| **Precision** | 0.5556     | 0.9444   | Male predictions are far more reliable. |
| **ROC AUC**   | 0.9583     | 0.9086   | Females benefit from stronger ranking quality despite lower precision. |
| **TPR**       | 0.8333     | 0.8854   | Sensitivity is slightly better for males. |

---

### 3. Interpretation  

- **Females (unprivileged):**  
  - Struggle with much lower **precision (0.5556)** and **F1-score (0.6667)**, meaning predictions are less reliable and effective.  
  - Have a **higher false positive rate (12.5%)**, causing more false alarms.  
  - ROC AUC (0.9583) is strong, showing the model ranks female cases well, but precision issues limit trust in predictions.  
  - Slightly lower TPR (0.8333) indicates more missed true cases.  

- **Males (privileged):**  
  - Achieve consistently better performance: higher accuracy, F1, and precision.  
  - Very strong precision (0.9444) and F1 (0.9140), with fewer false positives (10%).  
  - ROC AUC (0.9086) is slightly lower than females, but overall predictions for males are **much more reliable**.  

---

### **Summary**  
The KNN model introduces **clear disparities across genders**:  
- **Females** are disadvantaged by lower precision, weaker F1, and higher false positives, making predictions less trustworthy.  
- **Males** benefit from more accurate and reliable outcomes across nearly all performance metrics.  

While females show a strong ROC AUC, this advantage does not translate into practical reliability, confirming that KNN systematically **favors males in prediction quality**.  

---

In [15]:
#group specific error analysis

from fairmlhealth import performance_metrics as pm

# Define group masks with clear names
female_mask = (protected_attr_knn == 0)  # unprivileged group (female)
male_mask   = (protected_attr_knn == 1)  # privileged group (male)

# Function to evaluate group-specific metrics
def evaluate_group_performance(group_name, mask):
    tpr = pm.true_positive_rate(y_test[mask], y_pred_knn[mask])
    fpr = pm.false_positive_rate(y_test[mask], y_pred_knn[mask])
    print(f"{group_name} Results:")
    print(f"  True Positive Rate (TPR): {tpr:.4f}")
    print(f"  False Positive Rate (FPR): {fpr:.4f}")
    print("-" * 40)

# Evaluate for each group
evaluate_group_performance("Female (Unprivileged)", female_mask)
evaluate_group_performance("Male (Privileged)", male_mask)


Female (Unprivileged) Results:
  True Positive Rate (TPR): 0.8333
  False Positive Rate (FPR): 0.1250
----------------------------------------
Male (Privileged) Results:
  True Positive Rate (TPR): 0.8854
  False Positive Rate (FPR): 0.1000
----------------------------------------


### Group-Specific Error Analysis – KNN Model  

To further examine fairness at the subgroup level, we compared the **True Positive Rate (TPR)** and **False Positive Rate (FPR)** for the unprivileged (female) and privileged (male) groups.  

#### Results by Gender Group  

| Group                  | TPR     | FPR     |
|------------------------|---------|---------|
| Unprivileged (Female)  | 0.8333  | 0.1250  |
| Privileged (Male)      | 0.8854  | 0.1000  |

#### Interpretation  

- The **privileged group (male)** has a slightly higher **TPR (88.54%)** than the unprivileged group (83.33%), meaning males are somewhat more likely to be correctly identified when they truly have CVD.  
- The **FPR is lower for males (10.00%)** compared to females (12.50%), suggesting females are more often incorrectly flagged as having CVD.  
- Although the disparities are smaller than in some earlier cases, the model still shows **better sensitivity and specificity for males**, while females face relatively more false alarms and a higher risk of missed detections.  

#### Summary  

The subgroup-level results reveal a **persistent disadvantage for the unprivileged group (females)**.  
- They experience **slightly lower sensitivity** (TPR) and **higher false positive rates** than males.  
- These patterns confirm that even modest differences in error distribution can contribute to **systematic gender bias**, highlighting the importance of fairness-aware evaluation and potential bias mitigation.  

---

### Decision Tree - DT

In [16]:
import pandas as pd

# Load KNN results
dt_df = pd.read_csv("HeartFailureData_75M25F_BaselineDT_predictions.csv")

print(dt_df.head())

   Sex  y_true  y_prob  y_pred
0    1       1     1.0       1
1    1       1     0.0       0
2    1       1     1.0       1
3    1       1     0.0       0
4    0       0     0.0       0


In [17]:
import re

# Extract common columns
y_true_dt = dt_df["y_true"].values
y_prob_dt = dt_df["y_prob"].values
y_pred_dt = dt_df["y_pred"].values
gender_dt = dt_df["Sex"].values


# Use gender_knn as the protected attribute (0/1 as in your CSV)
protected_attr_dt = gender_dt


In [18]:
# Decision Tree Gender Bias Report
print("\n--- Decision Tree Gender Bias Report ---")

dt_bias = measure.summary(
    X=X_test,
    y_true=y_test,
    y_pred=y_pred_dt,
    y_prob=y_prob_dt,
    prtc_attr=protected_attr_dt,
    pred_type="classification",
    priv_grp=1,  # 1 = Male = Privileged
    sig_fig=4,
    skip_if=True,  
    skip_performance = True
)

print(dt_bias)


--- Decision Tree Gender Bias Report ---
                                                        Value
Metric         Measure                                       
Group Fairness AUC Difference                          0.0579
               Balanced Accuracy Difference            0.0579
               Balanced Accuracy Ratio                 1.0727
               Disparate Impact Ratio                  0.3885
               Equal Odds Difference                  -0.0950
               Equal Odds Ratio                        0.5682
               Positive Predictive Parity Difference  -0.3208
               Positive Predictive Parity Ratio        0.6339
               Statistical Parity Difference          -0.3727
Data Metrics   Prevalence of Privileged Class (%)     79.0000


In [19]:
#Flag metrics outside acceptable fairness bounds in current table 

styled_dt = MyFlagger().apply_flag(
    df=dt_bias,
    caption="Decision Tree Fairness (Gender)",
    boundaries=bounds,
    sig_fig=4,
    as_styler=True
)
styled_dt

## Gender Bias Detection Results – Decision Tree Model  

---

### 1. Group Fairness Metrics  

- **AUC Difference (0.0579):** The ROC AUC is moderately higher for one gender, suggesting better ranking ability for that group.  
- **Balanced Accuracy Difference (0.0579)** and **Ratio (1.0727):** Indicates that the model achieves somewhat better balanced accuracy for one gender, showing mild disparity.  
- **Disparate Impact Ratio (0.3885):** Well below the fairness threshold of 0.80–1.25, pointing to strong inequality in selection rates.  
- **Equal Odds Difference (−0.0950)** and **Equal Odds Ratio (0.5682):** Error rates (TPR and FPR) differ significantly across genders, favoring one group.  
- **Positive Predictive Parity Difference (−0.3208)** and **Ratio (0.6339):** Precision is much lower for one gender, showing predictions are less reliable for that subgroup.  
- **Statistical Parity Difference (−0.3727):** A substantial negative value, confirming that females are selected at a much lower rate than males.  

---

### 2. Interpretation  

- The Decision Tree model demonstrates **considerable fairness concerns**:  
  - While differences in **AUC and balanced accuracy** are modest, **disparate impact and statistical parity differences are severe**, indicating large selection imbalances.  
  - **Equal odds disparity** suggests that error rates are skewed, disadvantaging one group in terms of both false positives and false negatives.  
  - Precision (positive predictive value) is substantially worse for the disadvantaged group, lowering trust in positive predictions.  

---

### **Summary**  

The Decision Tree model shows **systematic gender bias**, with strong disparities in **selection rate, predictive precision, and error rates**.  
Although ranking ability (AUC) and balanced accuracy differences are relatively small, the large gaps in fairness metrics (disparate impact and statistical parity) highlight that the model **favors males while disadvantaging females**.  

---

In [20]:
print("FairMLHealth Stratified Bias Table - DT")
measure.bias(X_test, y_test, y_pred_dt, features=['Sex'], flag_oor=False)

FairMLHealth Stratified Bias Table - DT


,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,Sex,0,-0.0579,0.9322,0.095,1.7600,0.3208,1.5775,0.3727,2.5738,-0.0208,0.9750
1,Sex,1,0.0579,1.0727,-0.095,0.5682,-0.3208,0.6339,-0.3727,0.3885,0.0208,1.0256


## Stratified Bias Analysis – Decision Tree (DT) by Gender  

This table presents **group-specific fairness metrics** for the Decision Tree model, stratified by gender.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.  

---

### 1. Balanced Accuracy  
- **Females (0):** Difference = **−0.0579**, Ratio = **0.9322**  
- **Males (1):** Difference = **+0.0579**, Ratio = **1.0727**  
- ➝ The model achieves **better balanced accuracy for males**, while females face a noticeable disadvantage.  

---

### 2. False Positive Rate (FPR)  
- **Females (0):** FPR Difference = **+0.0950**, Ratio = **1.7600**  
- **Males (1):** FPR Difference = **−0.0950**, Ratio = **0.5682**  
- ➝ Females are **far more likely to be incorrectly classified as positive**, while males are less exposed to false positives.  

---

### 3. Positive Predictive Value (PPV / Precision)  
- **Females (0):** PPV Difference = **+0.3208**, Ratio = **1.5775**  
- **Males (1):** PPV Difference = **−0.3208**, Ratio = **0.6339**  
- ➝ Predictions are **more precise for females**, while males suffer from lower reliability of positive predictions.  

---

### 4. Selection Rate  
- **Females (0):** Selection Difference = **+0.3727**, Ratio = **2.5738**  
- **Males (1):** Selection Difference = **−0.3727**, Ratio = **0.3885**  
- ➝ The model **selects females much more frequently** than males, pointing to imbalance in decision allocation.  

---

### 5. True Positive Rate (TPR / Sensitivity)  
- **Females (0):** TPR Difference = **−0.0208**, Ratio = **0.9750**  
- **Males (1):** TPR Difference = **+0.0208**, Ratio = **1.0256**  
- ➝ Males have a **slight advantage in sensitivity**, meaning their true positive cases are more often detected.  

---

### **Summary**  
The Decision Tree model displays **mixed fairness outcomes**:  
- **Females** face a **higher false positive rate** but benefit from **higher precision and selection rates**.  
- **Males** enjoy **better balanced accuracy and slightly higher sensitivity (TPR)**, but their positive predictions are less precise and they are under-selected overall.  

Overall, the model exhibits **imbalances in both error rates and selection fairness**, suggesting that while females are more frequently selected, this comes at the cost of **more false alarms**, whereas males are somewhat overlooked but benefit from **lower misclassification risk**.  


---

In [21]:
# Get the stratified performance table
perf_table_dt = measure.performance(
    X=gender_df,
    y_true=y_test,
    y_pred=y_pred_dt,
    y_prob=y_prob_dt
)

# Replace NaN with a dash
perf_table_dt = perf_table_dt.fillna("—")

# Display pretty table
display(perf_table_dt)

C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\fairmlhealth\__utils.py:57: UserWarning: Possible error in column(s) ['Sex']. DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,Feature Name,Feature Value,Obs.,Mean Target,Mean Prediction,Accuracy,F1-Score,FPR,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,184.0,0.5543,0.5326,0.8152,0.8300,0.1829,0.3276,0.8469,0.8154,0.8137
1,Sex,0,38.0,0.1579,0.2368,0.8684,0.6667,0.1250,0.5497,0.5556,0.8542,0.8333
2,Sex,1,146.0,0.6575,0.6096,0.8014,0.8432,0.2200,0.2486,0.8764,0.7962,0.8125


## Stratified Performance Analysis – Decision Tree (DT) by Gender  

This table shows the **stratified performance metrics** of the Decision Tree model across gender groups.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.  

---

### 1. Overall Performance (All Features)  
- **Accuracy (0.8152)** and **F1-score (0.8300)** indicate moderate overall classification quality.  
- **ROC AUC (0.8154)** suggests decent but not excellent ability to distinguish between classes.  
- **Precision (0.8469)** is relatively high, meaning predictions are fairly reliable.  
- **True Positive Rate (TPR = 0.8137)** reflects good sensitivity overall, but subgroup breakdowns reveal disparities.  

---

### 2. Subgroup Comparison  

| Metric        | Female (0) | Male (1) | Interpretation |
|---------------|------------|----------|----------------|
| **Accuracy**  | 0.8684     | 0.8014   | Accuracy is higher for females than males. |
| **F1-Score**  | 0.6667     | 0.8432   | Model performs better for males in balancing precision and recall. |
| **FPR**       | 0.1250     | 0.2200   | Females face fewer false positives compared to males. |
| **Precision** | 0.5556     | 0.8764   | Predictions are much more reliable for males. |
| **ROC AUC**   | 0.8542     | 0.7962   | Females benefit from stronger ranking performance. |
| **TPR**       | 0.8333     | 0.8125   | Sensitivity is slightly higher for females. |

---

### 3. Interpretation  
- **Females (unprivileged):**  
  - Show **better accuracy and ROC AUC**, suggesting the model discriminates cases more effectively for this group.  
  - However, they experience **lower precision (0.5556)** and **weaker F1-score (0.6667)**, meaning their positive predictions are less reliable.  
  - They also benefit from a **lower false positive rate (12.5%)**, reducing the chance of being incorrectly flagged.  

- **Males (privileged):**  
  - Have **higher precision (0.8764) and stronger F1 (0.8432)**, meaning the model is more consistent in positive predictions for this group.  
  - However, they face a **higher false positive rate (22.0%)**, indicating more incorrect alarms.  
  - Their ROC AUC is lower, suggesting weaker ranking ability compared to females.  

---

### **Summary**  
The Decision Tree model demonstrates a **trade-off in gender-specific performance**:  
- **Females** benefit from **higher accuracy, ROC AUC, and lower false positives**, but their predictions are less precise and less balanced.  
- **Males** benefit from **greater precision and F1 performance**, but at the cost of **more false positives** and a weaker ROC AUC.  

This indicates that the model does not consistently favor one group but instead introduces **different types of disparities**, highlighting the need for fairness adjustments to balance error trade-offs across genders.  

---

In [22]:
from fairmlhealth import performance_metrics as pm

# Define group masks with clear names
female_mask = (protected_attr_dt == 0)  # female = unprivileged group
male_mask   = (protected_attr_dt == 1)  # male = privileged group 

# Function to evaluate group-specific metrics
def evaluate_group_performance(group_name, mask):
    tpr = pm.true_positive_rate(y_test[mask], y_pred_dt[mask])
    fpr = pm.false_positive_rate(y_test[mask], y_pred_dt[mask])
    print(f"{group_name} Results:")
    print(f"  True Positive Rate (TPR): {tpr:.4f}")
    print(f"  False Positive Rate (FPR): {fpr:.4f}")
    print("-" * 40)

# Evaluate for each group
evaluate_group_performance("Female (Unprivileged)", female_mask)
evaluate_group_performance("Male (Privileged)", male_mask)

Female (Unprivileged) Results:
  True Positive Rate (TPR): 0.8333
  False Positive Rate (FPR): 0.1250
----------------------------------------
Male (Privileged) Results:
  True Positive Rate (TPR): 0.8125
  False Positive Rate (FPR): 0.2200
----------------------------------------


### Group-Specific Error Analysis – Decision Tree Model  

To further examine fairness at the subgroup level, we compared the **True Positive Rate (TPR)** and **False Positive Rate (FPR)** for the unprivileged (female) and privileged (male) groups.  

#### Results by Gender Group  

| Group                  | TPR     | FPR     |
|------------------------|---------|---------|
| Unprivileged (Female)  | 0.8333  | 0.1250  |
| Privileged (Male)      | 0.8125  | 0.2200  |

#### Interpretation  

- The **True Positive Rate (TPR)** is slightly higher for females (83.33%) than for males (81.25%), meaning the model is marginally better at correctly detecting positive cases among females.  
- The **False Positive Rate (FPR)** is considerably lower for females (12.50%) compared to males (22.00%), indicating that males are more likely to be incorrectly flagged as positive cases.  
- Overall, this suggests the model is **both more sensitive and more specific for females**, while males face a greater risk of false alarms despite nearly equal sensitivity.  

#### Summary  

The Decision Tree model introduces a **systematic disadvantage for males (privileged group)**: although detection rates are nearly equal, males are far more prone to false positives. In contrast, females (unprivileged group) benefit from slightly higher sensitivity and significantly better specificity. This indicates that, unlike typical gender bias patterns, the model demonstrates a **reverse bias**, favoring females in error distribution.  

---

### Ensemble Model - Random Forest - RF

In [23]:
rf_df = pd.read_csv("HeartFailureData_75M25F_BaselineRF_predictions.csv")
print(rf_df.head())

   Sex  y_true  y_prob  y_pred
0    1       1    0.91       1
1    1       1    0.41       0
2    1       1    0.97       1
3    1       1    0.55       1
4    0       0    0.56       1


In [24]:
# Extract common columns
y_true_rf = rf_df["y_true"].values
y_pred_rf = rf_df["y_pred"].values
y_prob_rf = rf_df["y_prob"].values
gender_rf = rf_df["Sex"].values


# Use gender_knn as the protected attribute (0/1 as in your CSV)
protected_attr_rf = gender_rf

In [25]:
# Random Forest Gender Bias Report
print("\n--- Random Forest Gender Bias Report ---")

rf_bias = measure.summary(
    X=X_test,
    y_true=y_test,
    y_pred=y_pred_rf,
    y_prob=y_prob_rf,
    prtc_attr=protected_attr_rf,
    pred_type="classification",
    priv_grp=1,  # 1 = Male = Privileged
    sig_fig=4,
    skip_if=True,
    skip_performance = True
)

print(rf_bias)


--- Random Forest Gender Bias Report ---
                                                        Value
Metric         Measure                                       
Group Fairness AUC Difference                          0.0877
               Balanced Accuracy Difference            0.0792
               Balanced Accuracy Ratio                 1.0922
               Disparate Impact Ratio                  0.3921
               Equal Odds Difference                   0.0833
               Equal Odds Ratio                        0.6250
               Positive Predictive Parity Difference  -0.2980
               Positive Predictive Parity Ratio        0.6682
               Statistical Parity Difference          -0.4081
Data Metrics   Prevalence of Privileged Class (%)     79.0000


In [26]:
# Flagged fairness table for Random Forest
styled_rf = MyFlagger().apply_flag(
    df=rf_bias,
    caption="Random Forest Fairness (Gender)",
    boundaries=bounds,
    sig_fig=4,
    as_styler=True
)
styled_rf

## Gender Bias Detection Results for Random Forest Model  

---

### 1. Group Fairness Metrics  

- **AUC Difference (0.0877):** The ROC AUC is higher for one gender group, indicating better ranking performance compared to the other.  
- **Balanced Accuracy Difference (0.0792)** and **Ratio (1.0922):** Suggest that one gender benefits from notably better balanced accuracy.  
- **Disparate Impact Ratio (0.3921):** Far below the acceptable fairness threshold (0.80–1.25), highlighting **substantial inequality in selection rates**, strongly disadvantaging one gender.  
- **Equal Odds Difference (0.0833)** and **Equal Odds Ratio (0.6250):** Indicate meaningful disparities in error rates (TPR and FPR), with the model treating genders inconsistently.  
- **Positive Predictive Parity Difference (−0.2980)** and **Ratio (0.6682):** Predictions are less reliable for the unprivileged group, showing lower precision.  
- **Statistical Parity Difference (−0.4081):** Confirms a large imbalance in selection between genders, strongly disadvantaging the unprivileged group.  

---

### 2. Interpretation  

- The Random Forest model shows **clear evidence of gender bias**.  
- Females (unprivileged group) face lower **selection rates, precision, and statistical parity**, while males (privileged group) consistently benefit.  
- Disparities in **AUC and balanced accuracy** further confirm uneven predictive quality across genders.  
- The combination of **low disparate impact ratio and high statistical parity difference** signals a **systematic disadvantage for females**.  

---

### **Summary**  

The Random Forest model introduces **strong gender bias**.  
- **Females are disadvantaged**, experiencing reduced selection opportunities, less reliable predictions, and poorer error balance.  
- **Males are favored**, with higher predictive reliability and overall performance.  
- The fairness metrics suggest that Random Forest is **not equitable across genders** and requires bias mitigation before deployment.  

---

In [27]:
print("FairMLHealth Stratified Bias Table - RF")
measure.bias(X_test, y_test, y_pred_rf, features=['Sex'], flag_oor=False)

FairMLHealth Stratified Bias Table - RF


,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,Sex,0,-0.0792,0.9156,0.075,1.600,0.298,1.4966,0.4081,2.5507,-0.0833,0.9167
1,Sex,1,0.0792,1.0922,-0.075,0.625,-0.298,0.6682,-0.4081,0.3921,0.0833,1.0909


## Stratified Bias Analysis – Random Forest by Gender  

This table presents **group-specific fairness metrics** for the Random Forest model, stratified by gender.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.  

---

### 1. Balanced Accuracy  
- **Females (0):** Balanced Accuracy Difference = **−0.0792**, Ratio = **0.9156**  
- **Males (1):** Balanced Accuracy Difference = **+0.0792**, Ratio = **1.0922**  
- ➝ The model provides **higher balanced accuracy for males**, while females experience reduced predictive balance.  

---

### 2. False Positive Rate (FPR)  
- **Females (0):** FPR Difference = **+0.075**, Ratio = **1.600**  
- **Males (1):** FPR Difference = **−0.075**, Ratio = **0.625**  
- ➝ Females suffer from a **notably higher false positive rate**, being more often incorrectly flagged as positive cases.  

---

### 3. Positive Predictive Value (PPV / Precision)  
- **Females (0):** PPV Difference = **+0.298**, Ratio = **1.4966**  
- **Males (1):** PPV Difference = **−0.298**, Ratio = **0.6682**  
- ➝ Predictions are **more reliable for females** (higher precision), while males face reduced reliability.  

---

### 4. Selection Rate  
- **Females (0):** Selection Difference = **+0.4081**, Ratio = **2.5507**  
- **Males (1):** Selection Difference = **−0.4081**, Ratio = **0.3921**  
- ➝ Females are **selected far more frequently** than expected, while males are substantially under-selected.  

---

### 5. True Positive Rate (TPR / Sensitivity)  
- **Females (0):** TPR Difference = **−0.0833**, Ratio = **0.9167**  
- **Males (1):** TPR Difference = **+0.0833**, Ratio = **1.0909**  
- ➝ The model is **slightly more sensitive for males**, meaning their true cases are more often detected.  

---

### **Summary**  
- The Random Forest model displays **mixed bias patterns**:  
  - **Females** face disadvantages in **balanced accuracy** and **false positive rates**, but benefit from **higher precision and selection rates**.  
  - **Males** gain from **better sensitivity (TPR)** and **balanced accuracy**, but are **under-selected and predicted with lower reliability**.  
- These results suggest that Random Forest introduces **inconsistent fairness trade-offs**, with both genders experiencing different advantages and disadvantages.  

---

In [28]:
# Get the stratified performance table
perf_table_rf = measure.performance(
    X=gender_df,
    y_true=y_test,
    y_pred=y_pred_rf,
    y_prob=y_prob_rf
)

# Replace NaN with a dash
perf_table_rf = perf_table_rf.fillna("—")

# display pretty table
display(perf_table_rf)

C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\fairmlhealth\__utils.py:57: UserWarning: Possible error in column(s) ['Sex']. DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,Feature Name,Feature Value,Obs.,Mean Target,Mean Prediction,Accuracy,F1-Score,FPR,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,184.0,0.5543,0.5870,0.8804,0.8952,0.1707,—,0.8704,0.9186,0.9216
1,Sex,0,38.0,0.1579,0.2632,0.8947,0.7500,0.1250,—,0.6000,0.9818,1.0000
2,Sex,1,146.0,0.6575,0.6712,0.8767,0.9072,0.2000,—,0.8980,0.8941,0.9167


## Stratified Performance Analysis – Random Forest by Gender  

This table shows the **stratified performance metrics** of the Random Forest model across gender groups.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.  

---

### 1. Overall Performance (All Features)  
- **Accuracy (0.8804)** and **F1-score (0.8952)** indicate strong overall classification performance.  
- **ROC AUC (0.9186)** shows very good discriminatory ability.  
- **Precision (0.8704)** suggests predictions are generally reliable.  
- **TPR (0.9216)** demonstrates high sensitivity overall.  
- **Note**: PR AUC is not reported (“—”) due to dataset constraints.  

---

### 2. Subgroup Comparison  

| Metric        | Female (0) | Male (1) | Interpretation |
|---------------|------------|----------|----------------|
| **Accuracy**  | 0.8947     | 0.8767   | Accuracy is slightly higher for females. |
| **F1-Score**  | 0.7500     | 0.9072   | Males achieve much better balance between precision and recall. |
| **FPR**       | 0.1250     | 0.2000   | Males are more likely to be incorrectly flagged as positive (higher false positives). |
| **Precision** | 0.6000     | 0.8980   | Predictions for males are more reliable. |
| **ROC AUC**   | 0.9818     | 0.8941   | Females benefit from stronger ranking performance. |
| **TPR**       | 1.0000     | 0.9167   | Females have perfect sensitivity (no missed positives), while males have slightly lower sensitivity. |

---

### 3. Interpretation  
- **Females (unprivileged):**  
  - Benefit from **higher accuracy, ROC AUC, and perfect sensitivity (TPR = 1.0)**.  
  - However, they face **lower F1-score and precision**, meaning that while all true positives are caught, many false alarms reduce reliability.  

- **Males (privileged):**  
  - Achieve **higher F1 and precision**, showing more reliable predictions.  
  - However, they are disadvantaged by **higher false positive rates** and **lower ROC AUC**, indicating weaker ranking performance compared to females.  

---

### **Summary**  
The Random Forest model presents a **mixed fairness picture**:  
- **Females** gain from higher sensitivity and AUC, ensuring they are rarely missed.  
- **Males** gain from better reliability (precision and F1), but at the cost of more false positives.  

This indicates that the model distributes strengths and weaknesses differently across genders, leading to **trade-offs in fairness** rather than a clear bias toward one group.  

---

In [29]:
from fairmlhealth import performance_metrics as pm

# Define group masks with clear names
female_mask = (protected_attr_rf == 0)  # female = unprivileged group
male_mask   = (protected_attr_rf == 1)  # male = privileged group 

# Function to evaluate group-specific metrics
def evaluate_group_performance(group_name, mask):
    tpr = pm.true_positive_rate(y_test[mask], y_pred_rf[mask])
    fpr = pm.false_positive_rate(y_test[mask], y_pred_rf[mask])
    print(f"{group_name} Results:")
    print(f"  True Positive Rate (TPR): {tpr:.4f}")
    print(f"  False Positive Rate (FPR): {fpr:.4f}")
    print("-" * 40)

# Evaluate for each group
evaluate_group_performance("Female (Unprivileged)", female_mask)
evaluate_group_performance("Male (Privileged)", male_mask)

Female (Unprivileged) Results:
  True Positive Rate (TPR): 1.0000
  False Positive Rate (FPR): 0.1250
----------------------------------------
Male (Privileged) Results:
  True Positive Rate (TPR): 0.9167
  False Positive Rate (FPR): 0.2000
----------------------------------------


### Group-Specific Error Analysis – Random Forest Model  

To further examine fairness at the subgroup level, we compared the **True Positive Rate (TPR)** and **False Positive Rate (FPR)** for the unprivileged (female) and privileged (male) groups.  

#### Results by Gender Group  

| Group                  | TPR     | FPR     |
|------------------------|---------|---------|
| Unprivileged (Female)  | 1.0000  | 0.1250  |
| Privileged (Male)      | 0.9167  | 0.2000  |  

#### Interpretation  

- The **unprivileged group (female)** achieves a **perfect TPR (100%)**, meaning all true positive cases are correctly identified. However, their **FPR (12.5%)** indicates they are more likely to be incorrectly flagged as positive compared to an ideal outcome.  
- The **privileged group (male)** shows a **slightly lower TPR (91.67%)**, which means some true positive cases are missed, but they face a **higher FPR (20.0%)**, resulting in more false alarms compared to females.  
- This distribution suggests a **trade-off in errors**: females are guaranteed not to be missed (no false negatives), but at the cost of more false positives; males, on the other hand, benefit less from sensitivity but are disproportionately affected by false alarms.  

#### Summary  

The Random Forest model exhibits **asymmetric fairness trade-offs**:  
- **Females (unprivileged)**: advantaged in sensitivity (perfect TPR) but slightly disadvantaged in false positives.  
- **Males (privileged)**: disadvantaged in both sensitivity and false positives, leading to comparatively less favorable error distribution.  

This outcome shows that the model does **not consistently favor one gender**, but rather creates **imbalanced error profiles across groups**, raising fairness concerns that need to be addressed.  

---

### Deep Learning Model - Feed Forward Network (MLP)

In [30]:
mlp_df = pd.read_csv("HeartFailureData_75M25F_AdamMLP_predictions.csv")
print(mlp_df.head())

   Sex  y_true    y_prob  y_pred
0    1       1  0.951479       1
1    1       1  0.029132       0
2    1       1  0.810705       1
3    1       1  0.225562       0
4    0       0  0.612828       1


In [31]:
# Extract common columns 
y_true_mlp = mlp_df["y_true"].values 
y_prob_mlp = mlp_df["y_prob"].values
y_pred_mlp = mlp_df["y_pred"].values
gender_mlp = mlp_df["Sex"].values 

# Use gender_mlp as the protected attribute
protected_attr_mlp = gender_mlp 

In [32]:
#Run fairmlhealth bias detection for MLP 

mlp_bias = measure.summary(
    X=X_test,
    y_true=y_test,
    y_pred=y_pred_mlp,
    y_prob=y_prob_mlp,
    prtc_attr=protected_attr_mlp,
    pred_type="classification",
    priv_grp=1,
    sig_fig=4,
    skip_if=True,
    skip_performance = True
)

print(mlp_bias)

                                                        Value
Metric         Measure                                       
Group Fairness AUC Difference                          0.0646
               Balanced Accuracy Difference            0.0856
               Balanced Accuracy Ratio                 1.1005
               Disparate Impact Ratio                  0.4366
               Equal Odds Difference                   0.1562
               Equal Odds Ratio                        1.1852
               Positive Predictive Parity Difference  -0.3205
               Positive Predictive Parity Ratio        0.6519
               Statistical Parity Difference          -0.3396
Data Metrics   Prevalence of Privileged Class (%)     79.0000


In [33]:
# Flagged fairness table for MLP
styled_mlp = MyFlagger().apply_flag(
    df=mlp_bias,
    caption="MLP Fairness (Gender)",
    boundaries=bounds,
    sig_fig=4,
    as_styler=True
)
styled_mlp

## Fairness Evaluation – MLP by Gender

---

### 1. Group Fairness Metrics  

- **AUC Difference (0.0646):** Indicates a moderate disparity in ranking performance across genders, with one group benefiting from slightly better discriminative ability.  
- **Balanced Accuracy Difference (0.0856)** and **Ratio (1.1005):** Shows imbalances in sensitivity and specificity, with the model performing more favorably for one gender.  
- **Disparate Impact Ratio (0.4366):** Well below the fairness threshold of **0.80–1.25**, revealing a substantial imbalance in selection rates, strongly disadvantaging one group.  
- **Equal Odds Difference (0.1562)** and **Equal Odds Ratio (1.1852):** Reflect moderate disparities in error rates (TPR/FPR) across genders, though within the acceptable ratio range, indicating unequal error distribution.  
- **Positive Predictive Parity Difference (−0.3205)** and **Ratio (0.6519):** Precision is notably lower for one group, meaning predictions are significantly less reliable for them.  
- **Statistical Parity Difference (−0.3396):** Confirms that the disadvantaged group is selected far less often than the privileged group, reinforcing the presence of systematic bias.  

---

### 2. Interpretation  

- The MLP model exhibits **clear gender bias** across several fairness metrics.  
- Disparate Impact and Statistical Parity strongly indicate that one gender (most likely females as the unprivileged group) is **under-selected and disadvantaged**.  
- Error rate disparities (Equal Odds Difference) show that predictive performance is **not equally distributed**, with unprivileged individuals more likely to face unfair outcomes.  
- Lower predictive precision for the disadvantaged group further highlights a **consistency problem in prediction reliability**.  

---

### **Summary**  

The MLP model demonstrates **systematic bias against the unprivileged gender group**, reflected in:  
- **Lower selection rates** (DIR = 0.4366).  
- **Weaker precision and error rate fairness**.  
- **Moderate ranking performance disparities**.  

Overall, while the MLP achieves competitive accuracy, it introduces **significant fairness concerns**, particularly disadvantaging the unprivileged group in both selection outcomes and predictive reliability.  

---  


In [34]:
print("FairMLHealth Stratified Bias Table - MLP")
measure.bias(X_test, y_test, y_pred_mlp, features=['Sex'], flag_oor=False)

FairMLHealth Stratified Bias Table - MLP


,Feature Name,Feature Value,Balanced Accuracy Difference,Balanced Accuracy Ratio,FPR Diff,FPR Ratio,PPV Diff,PPV Ratio,Selection Diff,Selection Ratio,TPR Diff,TPR Ratio
0,Sex,0,-0.0856,0.9087,0.015,1.1200,0.3205,1.5341,0.3396,2.2904,-0.1562,0.8438
1,Sex,1,0.0856,1.1005,-0.015,0.8929,-0.3205,0.6519,-0.3396,0.4366,0.1562,1.1852


## Stratified Bias Analysis – MLP by Gender  

This table presents **group-specific fairness metrics** for the MLP model, stratified by gender.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.  

---

### 1. Balanced Accuracy  
- **Females (0):** Difference = **−0.0856**, Ratio = **0.9087**  
- **Males (1):** Difference = **+0.0856**, Ratio = **1.1005**  
- ➝ The model is **less accurate for females**, with better balanced accuracy for males.  

---

### 2. False Positive Rate (FPR)  
- **Females (0):** FPR Diff = **+0.015**, Ratio = **1.1200**  
- **Males (1):** FPR Diff = **−0.015**, Ratio = **0.8929**  
- ➝ Females experience a **slightly higher false positive rate**, meaning they are more often incorrectly flagged.  

---

### 3. Positive Predictive Value (PPV / Precision)  
- **Females (0):** PPV Diff = **+0.3205**, Ratio = **1.5341**  
- **Males (1):** PPV Diff = **−0.3205**, Ratio = **0.6519**  
- ➝ Predictions are **more reliable for females**, while males face reduced precision.  

---

### 4. Selection Rate  
- **Females (0):** Selection Diff = **+0.3396**, Ratio = **2.2904**  
- **Males (1):** Selection Diff = **−0.3396**, Ratio = **0.4366**  
- ➝ Females are **selected more often**, while males are **under-selected** by the model.  

---

### 5. True Positive Rate (TPR / Sensitivity)  
- **Females (0):** TPR Diff = **−0.1562**, Ratio = **0.8438**  
- **Males (1):** TPR Diff = **+0.1562**, Ratio = **1.1852**  
- ➝ The model detects **male cases more effectively**, with females facing a risk of missed true positives.  

---

### **Summary**  
The MLP model exhibits **mixed fairness outcomes**:  
- **Females** benefit from **higher selection rates and precision**, but face **more false positives** and a **lower true positive rate**.  
- **Males** are disadvantaged in terms of **precision and selection**, but benefit from **higher sensitivity and balanced accuracy**.  

Overall, the MLP introduces **trade-offs**: it favors females in terms of being selected and prediction reliability, but it **favors males in sensitivity and balanced accuracy**, leading to **asymmetric bias across metrics**.  

---

In [35]:
# Get the stratified performance table
perf_table_mlp = measure.performance(
    X=gender_df,
    y_true=y_test,
    y_pred=y_pred_mlp,
    y_prob=y_prob_mlp
)

# Replace NaN with a dash
perf_table_mlp = perf_table_mlp.fillna("—")

# display pretty table
display(perf_table_mlp)

C:\Users\patri\AppData\Roaming\Python\Python310\site-packages\fairmlhealth\__utils.py:57: UserWarning: Possible error in column(s) ['Sex']. DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.

  warn(f"Possible error in column(s) {cols}. {wr}\n")


,Feature Name,Feature Value,Obs.,Mean Target,Mean Prediction,Accuracy,F1-Score,FPR,PR AUC,Precision,ROC AUC,TPR
0,ALL FEATURES,ALL VALUES,184.0,0.5543,0.5326,0.8587,0.8700,0.1341,—,0.8878,0.9139,0.8529
1,Sex,0,38.0,0.1579,0.2632,0.8947,0.7500,0.1250,—,0.6000,0.9688,1.0000
2,Sex,1,146.0,0.6575,0.6027,0.8493,0.8804,0.1400,—,0.9205,0.9042,0.8438


## Stratified Performance Analysis – MLP by Gender  

This table shows the **stratified performance metrics** of the MLP model across gender groups.  
Here, **0 = Female (unprivileged)** and **1 = Male (privileged)**.  

---

### 1. Overall Performance (All Features)  
- **Accuracy (0.8587)** and **F1-score (0.8700)** suggest strong general predictive performance.  
- **ROC AUC (0.9139)** indicates very good discriminative ability.  
- **Precision (0.8878)** is high, reflecting reliable positive predictions.  
- **TPR (0.8529)** shows good sensitivity overall, though group disparities are present.  

---

### 2. Subgroup Comparison  

| Metric        | Female (0) | Male (1) | Interpretation |
|---------------|------------|----------|----------------|
| **Accuracy**  | 0.8947     | 0.8493   | Accuracy is higher for females, with males performing slightly worse. |
| **F1-Score**  | 0.7500     | 0.8804   | Males achieve better balance between precision and recall. |
| **FPR**       | 0.1250     | 0.1400   | False positives are slightly higher for males. |
| **Precision** | 0.6000     | 0.9205   | Predictions are much more reliable for males, while females face many false alarms. |
| **ROC AUC**   | 0.9688     | 0.9042   | Ranking performance is better for females. |
| **TPR**       | 1.0000     | 0.8438   | Sensitivity is perfect for females but weaker for males. |

---

### 3. Interpretation  

- **Females (unprivileged):**  
  - Benefit from **higher accuracy (89.5%)** and **perfect sensitivity (TPR = 1.0)**, meaning no true positive cases were missed.  
  - However, precision is **much lower (0.6000)**, showing a high false alarm rate despite strong detection ability.  

- **Males (privileged):**  
  - Have **higher precision (0.9205)** and a better F1-score, making their predictions more balanced and reliable.  
  - But sensitivity (TPR = 0.8438) is lower, meaning some true cases are missed compared to females.  

---

### **Summary**  

The MLP model exhibits **asymmetric performance trade-offs**:  
- **Females** are almost always correctly detected (high TPR) but with poor precision, leading to many false positives.  
- **Males** enjoy much more reliable predictions (high precision and F1) but face a **higher risk of missed detections**.  

This indicates that the MLP introduces **different types of bias**: females are over-flagged (false alarms), while males are under-detected (missed cases).  

---

In [36]:
from fairmlhealth import performance_metrics as pm

# Define group masks with clear names
female_mask = (protected_attr_mlp == 0)  # female = unprivileged group
male_mask   = (protected_attr_mlp == 1)  # male = privileged group 

# Function to evaluate group-specific metrics
def evaluate_group_performance(group_name, mask):
    tpr = pm.true_positive_rate(y_test[mask], y_pred_mlp[mask])
    fpr = pm.false_positive_rate(y_test[mask], y_pred_mlp[mask])
    print(f"{group_name} Results:")
    print(f"  True Positive Rate (TPR): {tpr:.4f}")
    print(f"  False Positive Rate (FPR): {fpr:.4f}")
    print("-" * 40)

# Evaluate for each group
evaluate_group_performance("Female (Unprivileged)", female_mask)
evaluate_group_performance("Male (Privileged)", male_mask)

Female (Unprivileged) Results:
  True Positive Rate (TPR): 1.0000
  False Positive Rate (FPR): 0.1250
----------------------------------------
Male (Privileged) Results:
  True Positive Rate (TPR): 0.8438
  False Positive Rate (FPR): 0.1400
----------------------------------------


### Group-Specific Error Analysis – MLP Model  

To further examine subgroup fairness, we compare the **True Positive Rate (TPR)** and **False Positive Rate (FPR)** between the unprivileged (female) and privileged (male) groups.  

#### Results by Gender Group  

| Group                  | TPR     | FPR     |
|------------------------|---------|---------|
| Unprivileged (Female)  | 1.0000  | 0.1250  |
| Privileged (Male)      | 0.8438  | 0.1400  |

#### Interpretation  

- **Females (unprivileged)** achieve a **perfect TPR (100%)**, meaning no positive cases were missed. However, their **FPR (12.5%)** indicates a moderate level of false alarms.  
- **Males (privileged)** show a **lower TPR (84.38%)**, meaning some positive cases are missed, while their **FPR (14.0%)** is slightly higher than for females, suggesting more false positives.  
- Overall, the model is **more sensitive for females**, detecting all true cases, but at the cost of moderate false positives. Males face both a **higher chance of being missed (lower TPR)** and **slightly more false alarms (higher FPR)**.  

#### Summary  

The MLP model introduces a **reverse bias pattern**, favoring **females (unprivileged group)** in terms of detection accuracy. Females benefit from **perfect sensitivity**, while males are disadvantaged by lower sensitivity and marginally higher error rates.  

---